## Imports Python

In [75]:
import pandas as pd
from datetime import time as dt_time
import datetime as datetime


## Classe AnalyzerGTFS

Cette classe a pour but de fouiller des données GTFS et d'en retourner les destinations possible en partant d'une ville, avec les différents trajets réalisables

In [99]:
# Version Stable de l'AnalyzerGTFS

# Exemple de ville : StopArea:OCE87586545

class AnalyzerGTFS:
    stop_time_trip = [] # Contient StopPoints, Trajets et horaires des trajets
    trip = [] # Contient les trip_id, route_id et les services_id
    stops_trip_id = [] # Contient tous les stops des trajets corrects
    date = [] # Contient tous les services et toutes les dates comprises entre date_min et date_max, (Utile pour afficher à l'utilisateur toutes les dates possibles pour le trajet)
    destinations = [] # Contient les stops après la ville (StopSequence > StopSequence de la ville)
    stops = []
    calendar_dates = []
    routes = []
    stop_times = []
    trips = []
    lat = 0
    lon = 0
    date_min = None
    date_max = None

    def __init__(self, path ='TER'):
        self.calendar_dates = pd.read_csv('Data/'+ path +'/calendar_dates.txt')
        self.stop_times = pd.read_csv('Data/'+ path +'/stop_times.txt')[['trip_id', 'stop_id', 'departure_time']]
        self.stops = pd.read_csv('Data/'+ path +'/stops.txt')[['stop_id', 'stop_name', 'stop_lat', 'stop_lon', 'parent_station']]
        self.trips = pd.read_csv('Data/'+ path +'/trips.txt')[['service_id', 'trip_id', 'route_id']]
        self.calendar_dates.date = pd.to_datetime(self.calendar_dates["date"], format='%Y%m%d').dt.date
        self.stops['parent_station'] = self.stops['parent_station'].fillna('')
    
    # Etape 1 : Récupérer les stops de la ville / StopArea
    def get_stops(self):
        self.id_ville = self.villes_proches(self.lat, self.lon)
        ville_Set = set(self.id_ville['stop_id'].array)
        return self.stops[[set([l]).issubset(ville_Set) for l in self.stops.parent_station.values.tolist()]]['stop_id']
    
    # Etape 2 : Récupérer tous les trajets des différents StopPoints, peut importe la date
    def get_trips(self):
        stops_ids = self.get_stops()
        self.stop_time_trip = pd.merge(self.stop_times, stops_ids, on='stop_id')
        self.stop_time_trip = self.stop_time_trip.drop_duplicates(subset='trip_id') #Doublons possibles si deux StopPoints différents sont sur un même trajet
        return self.stop_time_trip['trip_id']

    # Etape 3 : Récupérer les services_id des Trajets
    def get_service_id(self):
        trip_ids = self.get_trips()
        self.trip = pd.merge(trip_ids, self.trips, on='trip_id')
        return self.trip['service_id']
    
    # Etape 4 : Récupérer les services fonctionnant sur les dates données
    def get_dates(self):
        service_id = self.get_service_id()
        dates = pd.merge(service_id, self.calendar_dates, on='service_id')
        return dates[(dates['date'] >= self.date_min) & (dates['date'] <= self.date_max)]
    
    # Etape 5 : Récupérer les trajets des services fonctionnant sur les dates données
    def get_trajets(self):
        services_uniques = self.get_dates().drop_duplicates(subset='service_id')
        return pd.merge(services_uniques, self.trip, on='service_id')
    
    # Etape 6 : Récupérer les stops des trajets corrects
    def get_stops_trajets(self):
        trajets = self.get_trajets()
        return pd.merge(trajets, self.stop_times, on='trip_id')
    
    # Etape 7 : Récupérer les ids des destinations, c'est à dire les stops après la ville (StopSequence > StopSequence de la ville)
    def get_stops_destinations(self):
        stops_trip_id = self.get_stops_trajets().assign(temps_ville = "", stop_id_ville = "")
        stop_ids_ville = self.stop_time_trip['stop_id']
        stop_ids_ville.index = self.stop_time_trip['trip_id']
        stop_id_ville_origine = stop_ids_ville.loc[stops_trip_id['trip_id']]
        stops_trip_id['stop_id_ville'] = stop_id_ville_origine.array
        time = self.stop_time_trip['departure_time'] #StopTime trip
        time.index = self.stop_time_trip['trip_id']
        value = time.loc[stops_trip_id['trip_id']]
        stops_trip_id['temps_ville'] = value.array
        self.destinations = stops_trip_id[stops_trip_id['departure_time'] > stops_trip_id['temps_ville']]
        return self.destinations['stop_id']
    
    # Etape 8 : Récupérer les destinations
    def get_destinations(self):
        destinations_uniques = self.get_stops_destinations()
        destinations_StopPoint = pd.merge(destinations_uniques, self.stops, on='stop_id')
        destinationSet = set(destinations_StopPoint['parent_station'].array)
        destinations_StopArea = self.stops[[set([l]).issubset(destinationSet) for l in self.stops.stop_id.values.tolist()]]
        destinations_StopArea.drop_duplicates(subset='stop_id')
        return destinations_StopArea
    
    # Retourne les stopArea proche du point de départ
    def villes_proches(self,lat,long):
        return self.stops[(self.stops['stop_lat'] > lat-0.03) & (self.stops['stop_lat'] < lat+0.03) & (self.stops['stop_lon'] > long-0.03) & (self.stops['stop_lon'] < long+0.03) & self.stops['stop_id'].str.contains('StopArea')]
    
    # Pas très efficace, à améliorer
    def trajet_destination(self,villeDestination): 
        destinationsParentStation = pd.merge(self.destinations, self.stops, on='stop_id') # Merge entre les destinations et les stops pour récupérer les parent_station
        StopPoints = destinationsParentStation[villeDestination == destinationsParentStation['parent_station']] # Récupérer les StopPoints de la ville de destination
        trajets = pd.merge(StopPoints, self.destinations, on=['stop_id','service_id','temps_ville','departure_time','trip_id','stop_id_ville','route_id']) # Merge entre StopPoints de la ville et les destinations pour récupérer les trajets faisant le lien entre les deux
        trajets_services = pd.merge(trajets.drop_duplicates(subset='service_id'), self.get_dates(), on='service_id') 
        trajets_services.assign(jour_suivant_depart = 0, jour_suivant_arrivee = 0)
        '''
        trajets_services['jour_suivant_depart'] = trajets_services.apply(lambda x : x['temps_ville'] > datetime.datetime(hour = 24), axis=1)
        trajets_services['temps_ville'] = trajets_services.apply(lambda x : x['temps_ville']-datetime.datetime(hour = 24) if x['jour_suivant_depart'] else x['temps_ville'] , axis=1)
        trajets_services['jour_suivant_arrivee'] = trajets_services.apply(lambda x : x['departure_time'] > datetime.datetime(hour = 24), axis=1)
        trajets_services['temps_ville'] = trajets_services.apply(lambda x : x['departure_time']-datetime.datetime(hour = 24) if x['jour_suivant_arrivee'] else x['departure_time'] , axis=1)
        '''
        return trajets_services # Retourne tous les trajets faisant le lien entre la ville et la destination marchant sur les critères sélectionnés

    def list_of_cities(path):
        stops = pd.read_csv('Data/'+ path +'/stops.txt')[['stop_id', 'stop_name', 'stop_lat', 'stop_lon', 'parent_station']]
        return stops[stops['stop_id'].str.contains('StopArea')][['stop_name','stop_lat','stop_lon','stop_id']]
    
    def load_search(self, lat, lon ,date_min= datetime.datetime.today,date_max = datetime.datetime.today):
        self.lat = lat
        self.lon = lon
        self.date_min = datetime.datetime.strptime(date_min, '%Y%m%d').date()
        self.date_max = datetime.datetime.strptime(date_max, '%Y%m%d').date()

In [14]:
villeEx = 'StopArea:OCE87547000'

analyzer = AnalyzerGTFS(47.9,1.9,date_min='20240601', date_max= '20240731',path = 'INTERCITE')
AnalyzerGTFS.list_of_cities('INTERCITE')
#Analyzer.get_destinations()
#Analyzer.trajet_destination('StopArea:OCE87481002')
#Analyzer.villes_proches(47.9,1.9)

,stop_name,stop_lat,stop_lon,stop_id
0,Versailles Chantiers,48.795826,2.135883,StopArea:OCE87393009
2,Massy-Palaiseau,48.726421,2.257528,StopArea:OCE87393579
4,Chartres,48.448202,1.481313,StopArea:OCE87394007
6,Le Mans,47.995615,0.192614,StopArea:OCE87396002
8,Nantes,47.217491,-1.542077,StopArea:OCE87481002
...,...,...,...,...
289,Campagnac - Saint-Geniez,44.421956,3.066401,StopArea:OCE87783340
291,Sévérac-le-Château,44.324652,3.056945,StopArea:OCE87783365
293,Saint-Georges-de-Luzençon,44.066553,2.983784,StopArea:OCE87783423
295,Saint-Rome-de-Cernon,44.013981,2.967374,StopArea:OCE87783431


In [27]:
Test = analyzer.get_stops_trajets()
Test

,service_id,date,exception_type,trip_id,route_id,stop_id,departure_time
0,26,2024-06-26,1,OCESN3605F3066149:2024-06-26T00:35:36Z,FR:Line::276892b7-b6aa-4a9b-a989-62f4abea526b:,StopPoint:OCEINTERCITES-87547000,06:28:00
1,26,2024-06-26,1,OCESN3605F3066149:2024-06-26T00:35:36Z,FR:Line::276892b7-b6aa-4a9b-a989-62f4abea526b:,StopPoint:OCEINTERCITES-87543017,07:32:00
2,26,2024-06-26,1,OCESN3605F3066149:2024-06-26T00:35:36Z,FR:Line::276892b7-b6aa-4a9b-a989-62f4abea526b:,StopPoint:OCEINTERCITES-87576009,08:09:00
3,26,2024-06-26,1,OCESN3605F3066149:2024-06-26T00:35:36Z,FR:Line::276892b7-b6aa-4a9b-a989-62f4abea526b:,StopPoint:OCEINTERCITES-87597005,08:43:00
4,26,2024-06-26,1,OCESN3605F3066149:2024-06-26T00:35:36Z,FR:Line::276892b7-b6aa-4a9b-a989-62f4abea526b:,StopPoint:OCEINTERCITES-87592378,09:21:00
...,...,...,...,...,...,...,...
1748,215,2024-07-01,1,OCESN4085F3066626:2024-05-25T00:28:24Z,FR:Line::2381a54a-1f59-4452-88c7-303179dd472b:,StopPoint:OCEOUIGO-87574004,19:55:00
1749,215,2024-07-01,1,OCESN4085F3066626:2024-05-25T00:28:24Z,FR:Line::2381a54a-1f59-4452-88c7-303179dd472b:,StopPoint:OCEOUIGO-87571240,20:36:00
1750,215,2024-07-01,1,OCESN4085F3066626:2024-05-25T00:28:24Z,FR:Line::2381a54a-1f59-4452-88c7-303179dd472b:,StopPoint:OCEOUIGO-87487603,21:10:00
1751,215,2024-07-01,1,OCESN4085F3066626:2024-05-25T00:28:24Z,FR:Line::2381a54a-1f59-4452-88c7-303179dd472b:,StopPoint:OCEOUIGO-87484006,21:37:00


In [17]:
analyzer.stop_time_trip

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,OCESN3605F3066149:2024-06-26T00:35:36Z,07:30:00,07:32:00,StopPoint:OCEINTERCITES-87543017,1,NaN,0,0,NaN
1,OCESN3605F3165346:2024-06-26T00:35:36Z,07:28:00,07:30:00,StopPoint:OCEINTERCITES-87543017,1,NaN,0,0,NaN
2,OCESN3619F3064142:2024-06-26T00:35:36Z,09:22:00,09:24:00,StopPoint:OCEINTERCITES-87543017,1,NaN,0,1,NaN
3,OCESN3619F3064143:2024-05-15T00:29:57Z,09:22:00,09:24:00,StopPoint:OCEINTERCITES-87543017,1,NaN,0,1,NaN
4,OCESN3619F3066039:2024-04-16T00:31:58Z,09:27:00,09:29:00,StopPoint:OCEINTERCITES-87543017,1,NaN,0,1,NaN
...,...,...,...,...,...,...,...,...,...
264,OCESN4080F3330216:2024-06-19T00:38:12Z,09:31:00,09:34:00,StopPoint:OCEOUIGO-87543017,5,NaN,0,0,NaN
265,OCESN4082F3247560:2024-06-03T00:27:37Z,15:47:00,15:50:00,StopPoint:OCEOUIGO-87543017,5,NaN,0,0,NaN
266,OCESN4085F3021194:2024-06-26T00:35:36Z,19:25:00,19:28:00,StopPoint:OCEOUIGO-87543017,2,NaN,0,0,NaN
267,OCESN4085F3066626:2024-05-25T00:28:24Z,19:19:00,19:22:00,StopPoint:OCEOUIGO-87543017,2,NaN,0,0,NaN


In [63]:
'''
import folium

#city = Analyzer.stops.loc[Analyzer.stops['stop_id'] == villeEx]

map = folium.Map(location=[47.9,1.9], zoom_start=7)
#folium.CircleMarker([float(city.loc[:,['stop_lat']].values[0][0]), float(city.loc[:,['stop_lon']].values[0][0])], radius = 2, popup=city['stop_name'], color = "red").add_to(map)

for row in Destinations.itertuples():
    folium.CircleMarker([float(row.stop_lat), float(row.stop_lon)], radius = 2, popup=row.stop_name).add_to(map)
map
'''

In [28]:
stops_trip_id = analyzer.get_stops_trajets().assign(TempsVille = "", stop_id_ville = "")
stop_ids_ville = analyzer.stop_time_trip['stop_id']
stop_ids_ville.index = analyzer.stop_time_trip['trip_id']
stopIdVilleOrigine=stop_ids_ville.loc[stops_trip_id['trip_id']]
stops_trip_id['stop_id_ville'] = stopIdVilleOrigine.array
time = analyzer.stop_time_trip['departure_time'] #StopTime trip
time.index = analyzer.stop_time_trip['trip_id']
value=time.loc[stops_trip_id['trip_id']]
stops_trip_id['TempsVille'] = value.array
analyzer.destinations = stops_trip_id[stops_trip_id['departure_time'] > stops_trip_id['TempsVille']]
stops_trip_id

,service_id,date,exception_type,trip_id,route_id,stop_id,departure_time,TempsVille,stop_id_ville
0,26,2024-06-26,1,OCESN3605F3066149:2024-06-26T00:35:36Z,FR:Line::276892b7-b6aa-4a9b-a989-62f4abea526b:,StopPoint:OCEINTERCITES-87547000,06:28:00,07:32:00,StopPoint:OCEINTERCITES-87543017
1,26,2024-06-26,1,OCESN3605F3066149:2024-06-26T00:35:36Z,FR:Line::276892b7-b6aa-4a9b-a989-62f4abea526b:,StopPoint:OCEINTERCITES-87543017,07:32:00,07:32:00,StopPoint:OCEINTERCITES-87543017
2,26,2024-06-26,1,OCESN3605F3066149:2024-06-26T00:35:36Z,FR:Line::276892b7-b6aa-4a9b-a989-62f4abea526b:,StopPoint:OCEINTERCITES-87576009,08:09:00,07:32:00,StopPoint:OCEINTERCITES-87543017
3,26,2024-06-26,1,OCESN3605F3066149:2024-06-26T00:35:36Z,FR:Line::276892b7-b6aa-4a9b-a989-62f4abea526b:,StopPoint:OCEINTERCITES-87597005,08:43:00,07:32:00,StopPoint:OCEINTERCITES-87543017
4,26,2024-06-26,1,OCESN3605F3066149:2024-06-26T00:35:36Z,FR:Line::276892b7-b6aa-4a9b-a989-62f4abea526b:,StopPoint:OCEINTERCITES-87592378,09:21:00,07:32:00,StopPoint:OCEINTERCITES-87543017
...,...,...,...,...,...,...,...,...,...
1748,215,2024-07-01,1,OCESN4085F3066626:2024-05-25T00:28:24Z,FR:Line::2381a54a-1f59-4452-88c7-303179dd472b:,StopPoint:OCEOUIGO-87574004,19:55:00,19:22:00,StopPoint:OCEOUIGO-87543017
1749,215,2024-07-01,1,OCESN4085F3066626:2024-05-25T00:28:24Z,FR:Line::2381a54a-1f59-4452-88c7-303179dd472b:,StopPoint:OCEOUIGO-87571240,20:36:00,19:22:00,StopPoint:OCEOUIGO-87543017
1750,215,2024-07-01,1,OCESN4085F3066626:2024-05-25T00:28:24Z,FR:Line::2381a54a-1f59-4452-88c7-303179dd472b:,StopPoint:OCEOUIGO-87487603,21:10:00,19:22:00,StopPoint:OCEOUIGO-87543017
1751,215,2024-07-01,1,OCESN4085F3066626:2024-05-25T00:28:24Z,FR:Line::2381a54a-1f59-4452-88c7-303179dd472b:,StopPoint:OCEOUIGO-87484006,21:37:00,19:22:00,StopPoint:OCEOUIGO-87543017


In [44]:
'''
class AnalyzerGTFS2:
    stop_ids = [] #Identifiants des StopsPoints de la ville
    trip_ids = [] #Identifiants des Trajets liés à tous les StopPoints de la ville
    service_id = [] #Identifiants des services faisant des trajets à des dates comprises entre date_min et date_max
    stop_time_trip = [] # Contient StopPoints, Trajets et horaires des trajets
    trip = [] # Contient les trip_id, route_id et les services_id
    tripDates = []  # Contient les trajets des services fonctionnant sur les dates données
    stops_trip_id = [] # Contient tous les stops des trajets corrects
    date = [] # Contient tous les services et toutes les dates comprises entre date_min et date_max, (Utile pour afficher à l'utilisateur toutes les dates possibles pour le trajet)
    destinations = [] # Contient les stops après la ville (StopSequence > StopSequence de la ville)
    agency = pd.read_csv('Data/TER/agency.txt')
    feed_info = pd.read_csv('Data/TER/feed_info.txt')
    transfers = pd.read_csv('Data/TER/transfers.txt')
    calendar_dates = pd.read_csv('Data/TER/calendar_dates.txt')
    routes = pd.read_csv('Data/TER/routes.txt')
    stop_times = pd.read_csv('Data/TER/stop_times.txt')
    stops = pd.read_csv('Data/TER/stops.txt')
    trips = pd.read_csv('Data/TER/trips.txt')
    def __init__(self, id_ville, date_min,date_max):
        self.id_ville = id_ville
        self.date_min = datetime.strptime(date_min, '%Y%m%d').date()
        self.date_max = datetime.strptime(date_max, '%Y%m%d').date()
        self.stops.fillna('', inplace=True)
        self.calendar_dates.date = pd.to_datetime(self.calendar_dates["date"], format='%Y%m%d').dt.date
    
    # Etape 1 : Récupérer les stops de la ville / StopArea
    def get_stops(self):
        self.stop_ids = self.stops[self.stops['parent_station'].str.contains(self.id_ville)]['stop_id']
        return self.stop_ids
    
    # Etape 2 : Récupérer les trajets des StopPoints, peut importe la date
    def get_trips(self):
        self.stop_time_trip = pd.merge(self.stop_times, self.stop_ids, on='stop_id')
        self.trip_ids = self.stop_time_trip['trip_id']
        return self.trip_ids   

    # Etape 3 : Récupérer les services_id des Trajets
    def get_service_id(self):
        self.trip = pd.merge(self.trip_ids, self.trips, on='trip_id')
        self.service_id = self.trip['service_id']
        return self.service_id 
    
    # Etape 4 : Récupérer les services fonctionnant sur les dates données
    def get_dates(self):
        service_id = self.get_service_id()
        dates = pd.merge(service_id, self.calendar_dates, on='service_id')
        self.date = dates[(dates['date'] >= self.date_min) & (dates['date'] <= self.date_max)]
        return self.date
    
    # Etape 5 : Récupérer les trajets des services fonctionnant sur les dates données
    def get_trajets(self):
        services_uniques = self.date.drop_duplicates(subset='service_id')
        self.trajets = pd.merge(services_uniques, self.trip, on='service_id')
        return self.trajets
    
    # Etape 6 : Récupérer les stops des trajets corrects
    def get_stops_trajets(self):
        self.stops_trip_id = pd.merge(self.trajets, self.stop_times, on='trip_id')
        return self.stops_trip_id
    
    # Etape 7 : Récupérer les ids des destinations, c'est à dire les stops après la ville (StopSequence > StopSequence de la ville)
    def get_stops_destinations(self):
        Time = self.stop_time_trip['departure_time'] #StopTime trip
        Time.index = self.stop_time_trip['trip_id']

        self.stops_trip_id = self.stops_trip_id.assign(TempsVille = "")
        value=Time.loc[self.stops_trip_id['trip_id']]

        self.stops_trip_id['TempsVille'] = value.array

        self.destinations = self.stops_trip_id[self.stops_trip_id['departure_time'] > self.stops_trip_id['TempsVille']]
        return self.destinations
    
    # Etape 8 : Récupérer les destinations
    def get_destinations(self):
        #destinations_uniques = self.destinations.drop_duplicates(subset='stop_id')
        destinations_StopPoint = pd.merge(self.destinations, self.stops, on='stop_id')
        destinationSet = set(destinations_StopPoint['parent_station'].array)
        destinations_StopArea = self.stops[[set([l]).issubset(destinationSet) for l in self.stops.stop_id.values.tolist()]]
        destinations_StopArea.drop_duplicates(subset='stop_id')
        return destinations_StopArea
    
    def villes_proches(self,lat,long):
        return self.stops[(self.stops['stop_lat'] > lat-0.1) & (self.stops['stop_lat'] < lat+0.1) & (self.stops['stop_lon'] > long-0.1) & (self.stops['stop_lon'] < long+0.1) & self.stops['stop_id'].str.contains('StopArea')]
'''

In [45]:
'''
villeEx = 'StopArea:OCE87547000'

Analyzer2 = AnalyzerGTFS2(villeEx,'20240601','20240731')
Analyzer2.get_stops()
Analyzer2.get_trips()
Analyzer2.get_service_id()
Analyzer2.get_dates()
Analyzer2.get_trajets()
Analyzer2.get_stops_trajets()
Analyzer2.get_stops_destinations()
Analyzer2.get_destinations()
'''

C:\Users\thege\AppData\Local\Temp\ipykernel_22172\606114808.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  self.stops.fillna('', inplace=True)


,stop_id,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
4221,StopArea:OCE87543009,Orléans,,47.907891,1.904242,,,1,
4223,StopArea:OCE87543017,Les Aubrais,,47.926801,1.907129,,,1,
4226,StopArea:OCE87543033,Cercottes,,47.986155,1.884981,,,1,
4229,StopArea:OCE87543041,Chevilly,,48.026672,1.879113,,,1,
4232,StopArea:OCE87543058,Artenay,,48.081057,1.883226,,,1,
4235,StopArea:OCE87543066,Château-Gaillard,,48.141690,1.912815,,,1,
4238,StopArea:OCE87543074,Toury,,48.193606,1.939565,,,1,
4241,StopArea:OCE87543082,Boisseaux,,48.256418,1.973296,,,1,
4243,StopArea:OCE87543090,Angerville,,48.311632,2.003526,,,1,
4257,StopArea:OCE87543165,Salbris,,47.425186,2.047751,,,1,


In [2]:
import folium
from jinja2 import Template
from folium.map import Marker

# Modify Marker template to include the onClick event
click_template = """{% macro script(this, kwargs) %}
    var {{ this.get_name() }} = L.marker(
        {{ this.location|tojson }},
        {{ this.options|tojson }}
    ).addTo({{ this._parent.get_name() }}).on('click', onClick);
{% endmacro %}"""

# Change template to custom template
Marker._template = Template(click_template)

location_center = [51.7678, -0.00675564]
m = folium.Map(location_center, zoom_start=13)

# Create the onClick listener function as a branca element and add to the map html
click_js = """function onClick(e) {
                 var point = e.latlng; alert(point)
                 }"""
                 
e = folium.Element(click_js)
html = m.get_root()
html.script.get_root().render()
html.script._children[e.get_name()] = e

#Add marker (click on map an alert will display with latlng values)
marker = folium.Marker([51.7678, -0.00675564]).add_to(m)

m

In [114]:


date_to_timestamp('23:59:120')

86352

In [100]:
analyzer = AnalyzerGTFS('TER')
analyzer.load_search(42.4265,3.15805,'20240715','20240715')
villeDestination = 'StopArea:OCE87611004'
analyzer.get_destinations()

,stop_id,stop_name,stop_lat,stop_lon,parent_station
5232,StopArea:OCE87611004,Toulouse Matabiau,43.611206,1.453616,
5350,StopArea:OCE87611707,Escalquens,43.517093,1.542375,
5354,StopArea:OCE87611723,Baziège,43.453608,1.620698,
5392,StopArea:OCE87612002,Labège Innopole,43.547467,1.512964,
5394,StopArea:OCE87612010,Montaudran,43.573691,1.480201,
5468,StopArea:OCE87615013,Villefranche-de-Lauragais,43.398537,1.714725,
5470,StopArea:OCE87615039,Castelnaudary,43.312292,1.950390,
5472,StopArea:OCE87615054,Bram,43.243815,2.119325,
5474,StopArea:OCE87615112,Lézignan-Corbières,43.199956,2.769139,
5501,StopArea:OCE87615286,Carcassonne,43.218029,2.351824,


In [116]:
 

destinationsParentStation = pd.merge(analyzer.destinations, analyzer.stops, on='stop_id') # Merge entre les destinations et les stops pour récupérer les parent_station
StopPoints = destinationsParentStation[villeDestination == destinationsParentStation['parent_station']] # Récupérer les StopPoints de la ville de destination
trajets = pd.merge(StopPoints, analyzer.destinations, on=['stop_id','service_id','temps_ville','departure_time','trip_id','stop_id_ville','route_id']) # Merge entre StopPoints de la ville et les destinations pour récupérer les trajets faisant le lien entre les deux
trajets_services = pd.merge(trajets.drop_duplicates(subset='service_id'), analyzer.get_dates(), on='service_id') 
trajets_services = trajets_services.assign(jour_suivant_depart = "", jour_suivant_arrivee = "")

trajets_services['temps_ville'] = trajets_services.apply(lambda x : date_to_timestamp(x['temps_ville']), axis=1)
trajets_services['temps_ville']
# Rectification des temps de trajet si > 24h
trajets_services['jour_suivant_depart'] = trajets_services.apply(lambda x : x['temps_ville'] > 24*3600, axis=1)
trajets_services['temps_ville'] = trajets_services.apply(lambda x : x['temps_ville']-24*3600 if x['jour_suivant_depart'] else x['temps_ville'] , axis=1)
trajets_services['jour_suivant_arrivee'] = trajets_services.apply(lambda x : x['departure_time'] > 24*3600, axis=1)
trajets_services['departure_time'] = trajets_services.apply(lambda x : x['departure_time'] - 24*3600 if x['jour_suivant_depart'] else x['departure_time'], axis=1)

#trajets_services # Retourne tous les trajets faisant le lien entre la ville et la destination marchant sur les critères sélectionnés

0    26280
Name: temps_ville, dtype: int64